In [1]:
import os
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base="https://api.jarvis73.com/v1"
openai.organization = os.getenv("Organization ID")
#os.environ["OPENAI_API_KEY"] = ""

In [2]:
import t2ebm
import guidance

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from interpret.glassbox import ExplainableBoostingClassifier
from interpret import show

## 数据训练模型

In [3]:
df = pd.read_csv("./data/spaceship-titanic/train.csv") # replace with path to dataset on your machine
df.head()

dataset_description = ""
y_axis_description = ""

# transform cabin since 8000 unique values do not fit into the context windows of the LLM we want to use
df['Cabin'] = df['Cabin'].map(lambda x: x[:1] + '/' + x[-1] if isinstance(x, str) else x)

# pandas to numpy array
X_data = df.drop(columns=["PassengerId", "Transported", "Name"]).values
y_data = df["Transported"].values

X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)

In [4]:
dataset_description = """\nHi there, here is a general description of the data set on which I trained the model. This description is from kaggle:

Welcome to the year 2912, where your data science skills are needed to solve a cosmic mystery. We've received a transmission from four lightyears away and things aren't looking good.

The Spaceship Titanic was an interstellar passenger liner launched a month ago. With almost 13,000 passengers on board, the vessel set out on its maiden voyage transporting emigrants from our solar system to three newly habitable exoplanets orbiting nearby stars.

While rounding Alpha Centauri en route to its first destination—the torrid 55 Cancri E—the unwary Spaceship Titanic collided with a spacetime anomaly hidden within a dust cloud. Sadly, it met a similar fate as its namesake from 1000 years before. Though the ship stayed intact, almost half of the passengers were transported to an alternate dimension!

To help rescue crews and retrieve the lost passengers, you are challenged to predict which passengers were transported by the anomaly using records recovered from the spaceship’s damaged computer system.

The task is to predict whether a passenger was transported to an alternate dimension during the Spaceship Titanic's collision with the spacetime anomaly. To help you make these predictions, you're given a set of personal records recovered from the ship's damaged computer system.

Feature Descriptions:

PassengerId - A unique Id for each passenger. Each Id takes the form gggg_pp where gggg indicates a group the passenger is travelling with and pp is their number within the group. People in a group are often family members, but not always.
HomePlanet - The planet the passenger departed from, typically their planet of permanent residence.
CryoSleep - Indicates whether the passenger elected to be put into suspended animation for the duration of the voyage. Passengers in cryosleep are confined to their cabins.
Cabin - The cabin number where the passenger is staying. Takes the form deck/num/side, where side can be either P for Port or S for Starboard.
Destination - The planet the passenger will be debarking to.
Age - The age of the passenger.
VIP - Whether the passenger has paid for special VIP service during the voyage.
RoomService, FoodCourt, ShoppingMall, Spa, VRDeck - Amount the passenger has billed at each of the Spaceship Titanic's many luxury amenities.
Name - The first and last names of the passenger.
"""

y_axis_description = "The y-axis depicts contributions in log-odds towards the outcome, that is the probability that the passenger was transported to another dimension."

In [5]:
feature_names = df.drop(columns=["PassengerId", "Transported", "Name"]).columns.tolist()

ebm = ExplainableBoostingClassifier(interactions=0, 
                                    feature_names=feature_names)

ebm.fit(X_train, y_train)
ebm.score(X_test, y_test)

D:\anaconda3\lib\site-packages\interpret\glassbox\_ebm\_ebm.py:669: UserWarning: Missing values detected. Our visualizations do not currently display missing values. To retain the glassbox nature of the model you need to either set the missing values to an extreme value like -1000 that will be visible on the graphs, or manually examine the missing value score in ebm.term_scores_[term_index][0]
  warn(


0.78953421506613

In [13]:
def feature_importances_to_text(ebm):
    feature_importances = ""
    for feature_idx, feature_name in enumerate(ebm.feature_names_in_):   #得到数据的column： term imortance列表
        feature_importances += (
            f"{feature_name}: {ebm.term_importances()[feature_idx]:.2f}\n"
        )
    return feature_importances

In [150]:
X_test

array([['Mars', False, 'F/S', ..., 634.0, 3.0, 1057.0],
       ['Earth', False, 'G/S', ..., 0.0, 0.0, 1.0],
       ['Earth', True, 'G/S', ..., 0.0, 0.0, 0.0],
       ...,
       ['Earth', False, 'E/S', ..., 690.0, 260.0, 5.0],
       ['Mars', True, 'F/P', ..., 0.0, 0.0, 0.0],
       ['Earth', False, 'G/S', ..., 0.0, 1806.0, 0.0]], dtype=object)

## 函数llm2graph：提取ebm中某个feature的graph，然后通过llmchain来让llm根据该graph的信息对话。
1. 将ebm中的graph提取出来，转换成能传给LLM的文本格式
2. 将该格式加入prompt模板中并生成prompt
3. 根据生成的prompt建立LLMChain并返回

In [65]:
from langchain.agents import Tool
from langchain.agents import AgentType
from langchain.memory import ConversationBufferMemory 
from langchain import OpenAI, ConversationChain, LLMChain, PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.agents import initialize_agent

In [7]:
llm = ChatOpenAI(model_name= "gpt-3.5-turbo-16k",temperature=0)
llm_gpt4 = ChatOpenAI(model_name= "gpt-4",temperature=0)
memory = ConversationBufferMemory(memory_key="history")

In [9]:
def llm2graph(   
        llm,
        ebm,
        feature_idx,
        memory,
        num_sentences=7,
        dataset_description = None,
        y_axis_description = None,
):
    #返回一个chain，通过chain询问llm对序号为feature_idx的graph
    prefix = """You are an expert statistician and data scientist.
You interpret global explanations produced by a generalized additive model (GAM). GAMs produce explanations in the form of graphs that contain the effect of a specific input feature.\n
"""
    if dataset_description is None or dataset_description == '':
        prefix +="""You will be given graphs from the model, and the user will ask you questions about the graphs."""
    else:
        prefix +="""The user will first provide a general description of the dataset. Then you will be given graphs from the model, and the user will ask you questions about the graphs.\n"""
    
    prefix +="""\n\nAnswer all questions to the best of your ability, combining both the data contained in the graph"""
    
    if dataset_description is not None and len(dataset_description) > 0:
        prefix +=""", the data set description you were given, and your knowledge about the real world."""
    else:
        prefix +=""" and your knowledge about the real world."""
    
    prefix +="""Graphs will be presented as a JSON object with keys representing the x-axis and values representing the y-axis. For continuous features, the keys are intervals that represent ranges where the function predicts the same value. For categorical features, each key represents a possible value that the feature can take."""
    
    if y_axis_description is not None and len(y_axis_description) > 0:
        prefix +=y_axis_description
    prefix += """\n\nThe user will provide graphs in the following format:
        - The name of the feature depicted in the graph
        - The type of the feature (continuous, categorical, or boolean)
        - Mean values
        - Lower bounds of confidence interval
        - Upper bounds of confidence interval
    """
    if dataset_description is not None and len(dataset_description) > 0:
        prefix += dataset_description + """\nThe description of dataset ends.\n"""
    graph = t2ebm.graphs.extract_graph(ebm, feature_idx)
    graph = t2ebm.graphs.graph_to_text(graph)
    graph = graph.replace("{", "(").replace("}", ")")

    suffix="""\nBegin!
Human: Consider the following graph from the model.\n"""
    suffix+=graph
    suffix+="""\nAI: I have obtained the information of the graph. You can ask me questions next, and I will answer based on the information,my knowledge about the real world, and maybe the data description.
{history}
Human: {query}
AI:"""
    template=prefix+suffix
    prompt = PromptTemplate(input_variables=["history","query"], template=template)
    
    chain = LLMChain(
    llm = llm,
    prompt=prompt,
    verbose=False,
    memory=memory,
    )      
    return chain

In [10]:
#得到ebm的feature列表以及序号
for feature_idx, feature_name in enumerate(ebm.feature_names_in_):  
    print(f"{feature_idx}: {feature_name}  ")

0: HomePlanet  
1: CryoSleep  
2: Cabin  
3: Destination  
4: Age  
5: VIP  
6: RoomService  
7: FoodCourt  
8: ShoppingMall  
9: Spa  
10: VRDeck  


In [11]:
#生成一个关于0号feature的chain
graph = llm2graph(llm,ebm,0,memory=memory,dataset_description=dataset_description,y_axis_description=y_axis_description)

In [12]:
graph.run(query="Please describe the general pattern of the graph.")

'Based on the graph, we can see that the feature "HomePlanet" has three possible values: "Earth", "Europa", and "Mars". The graph shows the effect of each value on the probability of a passenger being transported to an alternate dimension. \n\nFrom the means, we can observe that passengers from "Europa" have the highest positive effect on the probability, followed by passengers from "Mars" with a smaller positive effect. On the other hand, passengers from "Earth" have a negative effect on the probability, indicating that they are less likely to be transported to an alternate dimension.\n\nThe confidence intervals provide a range of values within which the true effect of each value is likely to fall. For example, the lower bound of the confidence interval for "Europa" is 0.786, indicating that the true effect is likely to be at least this large. Similarly, the upper bound of the confidence interval for "Earth" is -0.407, indicating that the true effect is likely to be no larger than thi

In [102]:
graph.run(query="please provide a brief, at most 7 sentence summary of the influence of the feature on the outcome.")

'The HomePlanet feature has a significant influence on the probability of a passenger being transported to an alternate dimension during the collision with the spacetime anomaly. Passengers from "Europa" have the highest positive effect on the probability, indicating that they are more likely to be transported. Passengers from "Mars" also have a positive effect, but smaller than those from "Europa". On the other hand, passengers from "Earth" have a negative effect on the probability, suggesting that they are less likely to be transported. \n\nThese findings suggest that the planet of permanent residence plays a role in determining whether a passenger is affected by the anomaly. Passengers from "Europa" may have certain characteristics or factors that make them more susceptible to being transported, while passengers from "Earth" may have factors that make them less susceptible. The influence of the HomePlanet feature should be taken into account when predicting whether a passenger was t

In [103]:
graph.run(query="What will happen if I change one's 'HomePlanet' from 'Mars' to 'earth' ")

'Based on the graph and the information provided, changing a passenger\'s HomePlanet from "Mars" to "Earth" would result in a decrease in the probability of that passenger being transported to an alternate dimension. Passengers from "Mars" have a positive effect on the probability, while passengers from "Earth" have a negative effect. Therefore, changing the HomePlanet from "Mars" to "Earth" would likely decrease the probability of being transported to an alternate dimension. However, the exact magnitude of this change cannot be determined without additional information or the specific model coefficients.'

## 函数llm2ebm：对于整个ebm通过LLM进行对话
1. 通过函数llm2graph_desc获得每个feature的graph description
2. 将数据集描述，所有的feature importances，各个feature的graph description加入prompt中提供给llm
3. 根据prompt生成LLMChain，进行对话

In [44]:
#记忆模块
memory_ebm = ConversationBufferMemory(memory_key="history")

In [63]:
#返回某个feature的graph description
def llm2graph_desc(   
        llm,
        ebm,
        feature_idx,
        dataset_description = None,
        y_axis_description = None,
        query = "Now please provide a brief, at most 7 sentence summary of the influence of the feature on the outcome."
):
    prefix = """You are an expert statistician and data scientist.
You interpret global explanations produced by a generalized additive model (GAM). GAMs produce explanations in the form of graphs that contain the effect of a specific input feature.\n
"""
    if dataset_description is None or dataset_description == '':
        prefix +="""You will be given graphs from the model, and the user will ask you questions about the graphs."""
    else:
        prefix +="""The user will first provide a general description of the dataset. Then you will be given graphs from the model, and the user will ask you questions about the graphs.\n"""
    
    prefix +="""\n\nAnswer all questions to the best of your ability, combining both the data contained in the graph"""
    
    if dataset_description is not None and len(dataset_description) > 0:
        prefix +=""", the data set description you were given, and your knowledge about the real world."""
    else:
        prefix +=""" and your knowledge about the real world."""
    
    prefix +="""Graphs will be presented as a JSON object with keys representing the x-axis and values representing the y-axis. For continuous features, the keys are intervals that represent ranges where the function predicts the same value. For categorical features, each key represents a possible value that the feature can take."""
    
    if y_axis_description is not None and len(y_axis_description) > 0:
        prefix +=y_axis_description
    prefix += """\n\nThe user will provide graphs in the following format:
        - The name of the feature depicted in the graph
        - The type of the feature (continuous, categorical, or boolean)
        - Mean values
        - Lower bounds of confidence interval
        - Upper bounds of confidence interval
    """
    if dataset_description is not None and len(dataset_description) > 0:
        prefix += dataset_description + """\nThe description of dataset ends.\n"""
    graph = t2ebm.graphs.extract_graph(ebm, feature_idx)
    graph = t2ebm.graphs.graph_to_text(graph)
    graph = graph.replace("{", "(").replace("}", ")")

    suffix="""\nBegin!
Human: Consider the following graph from the model.\n"""
    suffix+=graph
    suffix+="""\nAI: I have obtained the information of the graph. You can ask me questions next, and I will answer based on the information,my knowledge about the real world, and maybe the data description.
Human: {query}
AI:"""
    template=prefix+suffix
    prompt = PromptTemplate(input_variables=["query"], template=template)
    
    chain = LLMChain(
    llm = llm,
    prompt=prompt,
    verbose=False,
    )
    graph_description = chain.run(query=query)
    return graph_description

In [67]:
ebm.feature_names_in_

['HomePlanet',
 'CryoSleep',
 'Cabin',
 'Destination',
 'Age',
 'VIP',
 'RoomService',
 'FoodCourt',
 'ShoppingMall',
 'Spa',
 'VRDeck']

In [51]:
#返回和ebm对话的LLMchain
def llm2ebm(   
        llm,
        ebm,
        memory,
        dataset_description = None,
        y_axis_description = None,
):
    feature_importances = feature_importances_to_text(ebm) 
    graphs = []
    graph_descriptions = []
    for feature_index in range(len(ebm.feature_names_in_)):       #获取ebm中的所有graph
        graphs.append(t2ebm.graphs.extract_graph(ebm, feature_index))
    graphs = [t2ebm.graphs.graph_to_text(graph) for graph in graphs]
    graph_descriptions = [llm2graph_desc(llm,ebm,idx,dataset_description=dataset_description,y_axis_description=y_axis_description) for idx in range(len(ebm.feature_names_in_)) ]
    graph_descriptions = "\n\n".join(
        [
            ebm.feature_names_in_[idx] + ": " + graph_description
            for idx, graph_description in enumerate(graph_descriptions)
        ]
    )
    
    prefix = """You are an expert statistician and data scientist.
            
    Your task is to provide an overall summary of a Generalized Additive Model (GAM) and answer the human's questions about it. The model consists of different graphs that contain the effect of a specific input feature.
    
    You will be given:
        - The global feature importances of the different features in the model.
        - Summaries of the graphs for the different features in the model. There is exactly one graph for each feature in the model.
    """
    if dataset_description is None or dataset_description == '':
        prefix += "\n\nThese inputs will be given to you by the user."
    else:
        prefix += "\n\nThe user will first provide a general description of what the dataset is about. Then you will be given the feature importance scores and the summaries of the individual features."
    
    suffix = ""
    
    if dataset_description is not None and len(dataset_description) > 0:
        suffix += "Human: Here is the general description of the data set\n" + dataset_description
        suffix += "\nAI: Thanks for this general description of the data set. Now please provide the global feature importance.\n"
    
    suffix += "Human: Here are the global feature importaces.\n\n" + feature_importances + "\nAI: Thanks. Now please provide the descriptions of the different graphs."
    suffix += "Human: Here are the descriptions of the different graphs.\n\n" + graph_descriptions
    suffix+="""\nAI: Thanks. You can ask me questions next.
    {history}
    Human: {query}
    AI:"""
    template=prefix+suffix
    prompt = PromptTemplate(input_variables=["history","query"], template=template)
    
    chain = LLMChain(
    llm = llm,
    prompt=prompt,
    verbose=False,
    memory=memory,
    )      
    return chain

In [52]:
#生成和ebm对话的LLMchain
talkebm = llm2ebm(llm,ebm,memory_ebm,dataset_description=dataset_description,y_axis_description=y_axis_description)

INFO: The graph of feature RoomService was simplified by 1.1%.
INFO: The graph of feature FoodCourt was simplified by 0.4%.
INFO: The graph of feature ShoppingMall was simplified by 0.4%.
INFO: The graph of feature Spa was simplified by 0.6%.
INFO: The graph of feature VRDeck was simplified by 0.6%.
INFO: The graph of feature RoomService was simplified by 1.1%.
INFO: The graph of feature FoodCourt was simplified by 0.4%.
INFO: The graph of feature ShoppingMall was simplified by 0.4%.
INFO: The graph of feature Spa was simplified by 0.6%.
INFO: The graph of feature VRDeck was simplified by 0.6%.


In [53]:
talkebm.run("Now give a summary about the model at most 10 sentences. Be sure to keep the most important information.")

'The Generalized Additive Model (GAM) was trained on a dataset from the Spaceship Titanic, which collided with a spacetime anomaly, transporting almost half of its passengers to an alternate dimension. The global feature importances indicate that the most influential features are Spa, VRDeck, CryoSleep, and RoomService. The graphs for each feature provide insights into their effects on the outcome. Passengers from the planet Europa have the highest probability of being transported, while those from Earth have the lowest. Choosing cryosleep increases the likelihood of being transported. Passengers staying in cabins labeled "C/S" and "C/P" are more likely to be transported, while those in "G/P" and "A/P" are less likely. The destination also plays a role, with passengers destined for "55 Cancri e" having a higher probability. Age, VIP status, and spending at amenities like the FoodCourt and ShoppingMall also influence the outcome. Overall, the GAM model provides valuable insights into th

In [54]:
talkebm.run("If a person's HomePlanet is Earth, CryoSleep is False, Cabin is F/4/S, Destination is TRAPPIST-1e, Age is 19.0, VIP is False, RoomService is 0.0, FoodCourt is 9.0, ShoppingMall is 0.0, Spa is 2823.0, VRDeck is 0.0, what is the probability that it is sent to the alternate dimension?")

'Based on the information provided, the probability that this person is sent to the alternate dimension can be estimated using the trained GAM model. However, without the specific coefficients and model parameters, it is not possible to provide an accurate probability estimate. To obtain a precise probability, the model would need to be applied to the given feature values using the appropriate calculations.'

In [55]:
talkebm.run("Can you predict based on the provided data description, feature importances and description of each graph?")

'Yes, based on the provided data description, feature importances, and descriptions of each graph, we can make a prediction about the probability of a person being sent to the alternate dimension. \n\nThe most influential features in the model are Spa, VRDeck, CryoSleep, and RoomService. However, we also need information about the specific coefficients and model parameters to make an accurate prediction. Without these details, it is not possible to provide a precise probability estimate. \n\nTo obtain a reliable prediction, the trained GAM model would need to be applied to the given feature values using the appropriate calculations.'

In [56]:
talkebm.run("You don't need to make a very precise prediction, you just need to get a probability based on the information I provided. Please tell me the probability of this person being transported to the alternate dimension.")

'Based on the information provided, the probability of this person being transported to the alternate dimension cannot be accurately determined without the specific coefficients and model parameters. To obtain a precise probability estimate, the trained GAM model would need to be applied to the given feature values using the appropriate calculations.'

In [58]:
talkebm.run("If a person's RoomService changes from 0 to 50, and ShoppingMall changes from 0 to 20, what impact will it have on the probability of being sent to the alternate dimension?")

'Based on the descriptions of the RoomService and ShoppingMall graphs, increasing the value of RoomService from 0 to 50 and ShoppingMall from 0 to 20 would likely have an impact on the probability of being sent to the alternate dimension. \n\nFor RoomService, as the amount billed increases, the probability of being transported to an alternate dimension also increases up to a certain point. Therefore, increasing the value of RoomService from 0 to 50 may increase the probability of being sent to the alternate dimension.\n\nFor ShoppingMall, the graph suggests that higher spending at the ShoppingMall is associated with a higher probability of being transported to an alternate dimension. Therefore, increasing the value of ShoppingMall from 0 to 20 may also increase the probability of being sent to the alternate dimension.\n\nHowever, without the specific coefficients and model parameters, it is not possible to provide an accurate estimate of the impact on the probability. To obtain a preci

In [59]:
#使用gpt4
talkebm = llm2ebm(llm_gpt4,ebm,memory_ebm,dataset_description=dataset_description,y_axis_description=y_axis_description)

INFO: The graph of feature RoomService was simplified by 1.1%.
INFO: The graph of feature FoodCourt was simplified by 0.4%.
INFO: The graph of feature ShoppingMall was simplified by 0.4%.
INFO: The graph of feature Spa was simplified by 0.6%.
INFO: The graph of feature VRDeck was simplified by 0.6%.
INFO: The graph of feature RoomService was simplified by 1.1%.
INFO: The graph of feature FoodCourt was simplified by 0.4%.
INFO: The graph of feature ShoppingMall was simplified by 0.4%.
INFO: The graph of feature Spa was simplified by 0.6%.
INFO: The graph of feature VRDeck was simplified by 0.6%.


In [60]:
talkebm.run("If a person's HomePlanet is Earth, CryoSleep is False, Cabin is F/4/S, Destination is TRAPPIST-1e, Age is 19.0, VIP is False, RoomService is 0.0, FoodCourt is 9.0, ShoppingMall is 0.0, Spa is 2823.0, VRDeck is 0.0, what is the probability that it is sent to the alternate dimension?")

'Based on the descriptions of the graphs for each feature, we can make a qualitative prediction. \n\nThe person is from Earth, which has a negative contribution to the log-odds of being transported to an alternate dimension. They are not in CryoSleep, which also contributes negatively. The cabin type is not specified in the graph descriptions, so its effect is unclear. The destination, TRAPPIST-1e, contributes negatively. Being 19 years old seems to have a negative contribution as well, as the age graph shows a decrease in log-odds for this age group. The person is not a VIP, which has a near-zero contribution. RoomService is 0, which is in the range that contributes positively. FoodCourt is 9, which is not clearly specified in the graph description, but lower values generally contribute negatively. ShoppingMall is 0, which contributes negatively. Spa is 2823, which is likely to contribute negatively as higher values of Spa usage are associated with a lower likelihood of being transpor

In [61]:
talkebm.run("If his Spa changes from 2823 to 20 and HomePlanet changes from Earth to Mars, what impact will it have on his probability of being teleported to the alternate dimension?")

"Based on the descriptions of the graphs for each feature, we can make a qualitative prediction. \n\nThe person's HomePlanet changing from Earth to Mars would likely increase the probability of being transported to an alternate dimension. This is because the graph for HomePlanet shows that passengers from Mars have a slight positive contribution to the log-odds of being transported, while passengers from Earth have a negative contribution.\n\nThe person's Spa usage changing from 2823 to 20 would also likely increase the probability of being transported to an alternate dimension. This is because the graph for Spa shows that lower values of Spa usage contribute positively to the log-odds of being transported, while higher values contribute negatively.\n\nHowever, without the specific coefficients and model parameters, it is not possible to provide an accurate estimate of the impact on the probability. To obtain a precise estimate, the trained GAM model would need to be applied to the giv

## Agent

In [18]:
from langchain.agents import load_tools
from langchain.agents import ZeroShotAgent
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMMathChain
from langchain.agents import AgentType, initialize_agent
from langchain.tools import BaseTool, StructuredTool, Tool, tool
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import BaseChatPromptTemplate
from langchain.chains.llm import LLMChain
from langchain.memory import ConversationBufferMemory
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish, HumanMessage
import re

In [23]:
llm = ChatOpenAI(model_name= "gpt-3.5-turbo-16k",temperature=0)
llm_gpt4 = ChatOpenAI(model_name= "gpt-4",temperature=0)

In [ ]:
#part of tool description
"""
"For example, the question is if a persion's Age is 27,WorkClass is Private,fnlwgt is 160178,Education "
    "is Some-college, EducationNum is 10, MaritalStatus is Divorced ,Occupation is Adm-clerical,what is the prediction."
    "you need to provide this tool with [27,' Private', 160178,' Some-college', 10, ' Divorced', ' Adm-clerical']"


    "For example, the question is if a person's HomePlanet is Earth, CryoSleep is False, Cabin is F/4/S,"
    " Destination is TRAPPIST-1e, Age is 19.0, VIP is False, RoomService is 0.0, FoodCourt"
    " is 9.0, ShoppingMall is 0.0, Spa is 2823.0, VRDeck is 0.0, what is the probability"
    " that it is sent to the alternate dimension? you need to provide this tool with "
    "['Earth','False','F/4/S','TRAPPIST-1e',19.0,False,0.0,9.0,0.0,2823.0,0.0]."
"""

In [299]:
desc = (
    "use this tool when you need to predict the probabilities given a series of feature values"
    "in a Generalized Additive Model. It will return the final probability.To use the tool"
    " you must provide a list consisting of the value of each feature.The value must be provided in question."
    "For example, the question is if a persion's Age is 27,WorkClass is Private,fnlwgt is 160178,Education "
    "is Some-college, EducationNum is 10, MaritalStatus is Divorced ,Occupation is Adm-clerical,what is the prediction."
    "you need to provide this tool with [27,' Private', 160178,' Some-college', 10, ' Divorced', ' Adm-clerical']"
    #"['adjacent_side', 'opposite_side', 'angle']."
)

In [300]:
def forecast(input: np.ndarray):
    sample = eval(input)
    return ebm.predict(sample)

In [301]:
forecast_tool = Tool(
    name='Forecast',
    func=forecast,
    description=desc
)

In [302]:
tools=[]
tools.append(forecast_tool)

In [304]:
#和ebm对话的prompt生成
feature_importances = feature_importances_to_text(ebm) 
graphs = []
graph_descriptions = []
for feature_index in range(len(ebm.feature_names_in_)):       #获取ebm中的所有graph
    graphs.append(t2ebm.graphs.extract_graph(ebm, feature_index))
graphs = [t2ebm.graphs.graph_to_text(graph) for graph in graphs]
'''graph_descriptions = [llm2graph_desc(llm,ebm,idx,dataset_description=dataset_description,y_axis_description=y_axis_description) for idx in range(len(ebm.feature_names_in_)) ]
graph_descriptions = "\n\n".join(
    [
        ebm.feature_names_in_[idx] + ": " + graph_description
        for idx, graph_description in enumerate(graph_descriptions)
    ]
)'''

prefix = """You are an expert statistician and data scientist.
        
Your task is complete some tasks about a Generalized Additive Model (GAM). The model consists of different graphs that contain the effect of a specific input feature.
You are bad at math such as predict the probability. So you must directly use the tool when asked about math problem. 

You will be given:
    - The global feature importances of the different features in the model.
    - Summaries of the graphs for the different features in the model. There is exactly one graph for each feature in the model.
"""
if dataset_description is None or dataset_description == '':
    prefix += "\n\nThese inputs will be given to you by the user."
else:
    prefix += "\n\nThe user will first provide a general description of what the dataset is about. Then you will be given the feature importance scores and the summaries of the individual features."

suffix = ""

if dataset_description is not None and len(dataset_description) > 0:
    suffix += "Here is the general description of the data set\n" + dataset_description
    
suffix += "\nHere are the global feature importances.Be sure not to provide these importances for the tool directly for prediction.\n\n" + feature_importances 
#suffix += "\nHere are the descriptions of the different graphs.\n\n"
#suffix += graph_descriptions
suffix+="""
Complete the objective as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of {tool_names}
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Previous conversation history:
{history}

Question: {input}
{agent_scratchpad}"""
template = prefix + suffix

INFO: The graph of feature RoomService was simplified by 1.1%.
INFO: The graph of feature FoodCourt was simplified by 0.4%.
INFO: The graph of feature ShoppingMall was simplified by 0.4%.
INFO: The graph of feature Spa was simplified by 0.6%.
INFO: The graph of feature VRDeck was simplified by 0.6%.


In [305]:
# Set up a prompt template 按照定义的 template，将需要的值都填写进去
class CustomPromptTemplate(BaseChatPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]
    
    def format_messages(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        formatted = self.template.format(**kwargs)
        return [HumanMessage(content=formatted)]

In [306]:
prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps","history"]
)

In [307]:
memory_ag = ConversationBufferMemory(memory_key="history")

In [308]:
#解析 llm 的输出，根据输出文本找到需要执行的决策。
class CustomOutputParser(AgentOutputParser):
    
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            new_out = llm_output.split("Action:")[0]
            return AgentFinish(
                return_values={"output": new_out.split(":")[1].strip()},
                log=llm_output,
            )
            #raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

In [309]:
output_parser = CustomOutputParser()

In [310]:
# LLM chain consisting of the LLM and a prompt
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [311]:
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=output_parser,
    stop=["\nObservation:"], 
    allowed_tools=tool_names
)

In [312]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, memory=memory_ag ,verbose=True)

In [72]:
agent_executor.run("Now give a summary about the model at most 10 sentences. Be sure to keep the most important information.")



> Entering new AgentExecutor chain...
Thought: To summarize the model, I need to highlight the important information about the Generalized Additive Model (GAM) based on the given feature importances and summaries of the individual features.

Action: I will start by mentioning that the GAM consists of different graphs that represent the effect of each input feature on the prediction. The feature importances indicate the relative importance of each feature in the model, with Spa having the highest importance (0.75) followed by VRDeck (0.66). CryoSleep (0.52) and RoomService (0.52) also have relatively high importances.

Action Input: None


Observation:I will start by mentioning that the GAM consists of different graphs that represent the effect of each input feature on the prediction. The feature importances indicate the relative importance of each feature in the model, with Spa having the highest importance (0.75) followed by VRDeck (0.66). CryoSleep (0.52) and RoomService (0.52) als

'The Generalized Additive Model (GAM) consists of different graphs representing the effect of each input feature on the prediction. The feature importances indicate the relative importance of each feature, with Spa and VRDeck having the highest importances. The individual feature summaries show the relationships between each feature and the prediction, with some features having non-linear relationships.'

In [313]:
agent_executor.run("If a person's HomePlanet is Earth, CryoSleep is False, Cabin is F/4/S, Destination is TRAPPIST-1e, Age is 19.0, VIP is False, RoomService is 0.0, FoodCourt is 9.0, ShoppingMall is 0.0, Spa is 2823.0, VRDeck is 0.0, what is the probability that it is sent to the alternate dimension?")



> Entering new AgentExecutor chain...
Thought: We need to use the Forecast tool to predict the probability.

Action: Forecast
Action Input: ['Earth', False, 'F/4/S', 'TRAPPIST-1e', 19.0, False, 0.0, 9.0, 0.0, 2823.0, 0.0]

Observation:[False]
The probability that the person is sent to the alternate dimension is False.

Final Answer: False

> Finished chain.


'False'

In [314]:
agent_executor.run("what is the answer of the last question?")



> Entering new AgentExecutor chain...
Thought: The answer to the last question is False.

Final Answer: False

> Finished chain.


'False'

In [283]:
#显示agent的prompt template
print(agent.llm_chain.prompt.template)

You are an expert statistician and data scientist.
        
Your task is complete some tasks about a Generalized Additive Model (GAM). The model consists of different graphs that contain the effect of a specific input feature.
You are bad at math such as predict the probability. So you must directly use the tool when asked about math problem. 

You will be given:
    - The global feature importances of the different features in the model.
    - Summaries of the graphs for the different features in the model. There is exactly one graph for each feature in the model.


The user will first provide a general description of what the dataset is about. Then you will be given the feature importance scores and the summaries of the individual features.Here is the general description of the data set

Hi there, here is a general description of the data set on which I trained the model. This description is from kaggle:

Welcome to the year 2912, where your data science skills are needed to solve a 

In [141]:
#和ebm对话的prompt生成
feature_importances = feature_importances_to_text(ebm) 
graphs = []
graph_descriptions = []
for feature_index in range(len(ebm.feature_names_in_)):       #获取ebm中的所有graph
    graphs.append(t2ebm.graphs.extract_graph(ebm, feature_index))
graphs = [t2ebm.graphs.graph_to_text(graph) for graph in graphs]

prefix = """You are an expert statistician and data scientist.
        
Your task is to provide an overall summary of a Generalized Additive Model (GAM). The model consists of different graphs that contain the effect of a specific input feature.

You will be given:
    - The global feature importances of the different features in the model.
    - Summaries of the graphs for the different features in the model. There is exactly one graph for each feature in the model.
"""
if dataset_description is None or dataset_description == '':
    prefix += "\n\nThese inputs will be given to you by the user."
else:
    prefix += "\n\nThe user will first provide a general description of what the dataset is about. Then you will be given the feature importance scores and the summaries of the individual features."

suffix = ""

if dataset_description is not None and len(dataset_description) > 0:
    suffix += "Here is the general description of the data set\n" + dataset_description
    
suffix += "Here are the global feature importaces.\n\n" + feature_importances 
suffix += "\nHere are the descriptions of the different graphs.\n\n" + "You have access to the following tools:"
prefix = prefix + suffix
suffix="""
Begin!
{chat_history}
Question: {input}
{agent_scratchpad}"""

INFO: The graph of feature RoomService was simplified by 1.1%.
INFO: The graph of feature FoodCourt was simplified by 0.4%.
INFO: The graph of feature ShoppingMall was simplified by 0.4%.
INFO: The graph of feature Spa was simplified by 0.6%.
INFO: The graph of feature VRDeck was simplified by 0.6%.


In [131]:
tools = [
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math",
    )
]

In [142]:
prompt2 = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "chat_history", "agent_scratchpad"],
)
memory = ConversationBufferMemory(memory_key="chat_history")

In [145]:
llm_chain = LLMChain(llm=ChatOpenAI(temperature=0), prompt=prompt)
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)
agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True, memory=memory
)

In [ ]:
agent_chain.run(input="If a person's HomePlanet is Earth, CryoSleep is False, Cabin is F/4/S, Destination is TRAPPIST-1e, Age is 19.0, VIP is False, RoomService is 0.0, FoodCourt is 9.0, ShoppingMall is 0.0, Spa is 2823.0, VRDeck is 0.0, what is the probability that it is sent to the alternate dimension?")

## Agent test

In [149]:
from langchain.agents import tool
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

tools = [get_word_length]

ImportError: cannot import name 'AgentActionMessageLog' from 'langchain.schema.agent' (D:\anaconda3\lib\site-packages\langchain\schema\agent.py)

In [150]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are very powerful assistant, but bad at calculating lengths of words."),
    MessagesPlaceholder(variable_name=MEMORY_KEY),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

In [113]:
from langchain.schema.messages import HumanMessage, AIMessage
chat_history = []

In [147]:
from langchain.tools.render import format_tool_to_openai_function
llm_with_tools = llm.bind(
    functions=[format_tool_to_openai_function(t) for t in tools]
)

In [148]:
agent = {
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_to_openai_functions(x['intermediate_steps']),
    "chat_history": lambda x: x["chat_history"]
} | prompt | llm_with_tools | OpenAIFunctionsAgentOutputParser()
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

NameError: name 'OpenAIFunctionsAgentOutputParser' is not defined

In [ ]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True，memory=memory)

In [ ]:
agent_executor.invoke({"input": "how many letters in the word educa?"})

NameError: name 'chat_history' is not defined

In [99]:
from langchain.chains import LLMMathChain
from langchain.llms import OpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.utilities import SQLDatabase
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI

In [100]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
tools = [
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math",
    ),
]

In [101]:
from langchain.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory

agent_kwargs = {
    "extra_prompt_messages": [MessagesPlaceholder(variable_name="memory")],
}
memory = ConversationBufferMemory(memory_key="memory", return_messages=True)

In [102]:
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
    agent_kwargs=agent_kwargs,
    memory=memory,
    prompt=prompt
)

In [108]:
agent.agent.prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input', 'memory'], output_parser=None, partial_variables={}, messages=[SystemMessage(content='You are a helpful AI assistant.', additional_kwargs={}), MessagesPlaceholder(variable_name='memory'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], output_parser=None, partial_variables={}, template='{input}', template_format='f-string', validate_template=True), additional_kwargs={}), MessagesPlaceholder(variable_name='agent_scratchpad')])

AttributeError: 'OpenAIFunctionsAgent' object has no attribute 'llm_chain'

## test

In [18]:
#和ebm对话的prompt生成

graphs = []
graph_descriptions = []
for feature_index in range(len(ebm.feature_names_in_)):       #获取ebm中的所有graph
    graphs.append(t2ebm.graphs.extract_graph(ebm, feature_index))
graphs = [t2ebm.graphs.graph_to_text(graph) for graph in graphs]
#graph_descriptions = [llm2graph_desc(llm,ebm,idx,dataset_description=dataset_description,y_axis_description=y_axis_description) for idx in range(len(ebm.feature_names_in_)) ]


prefix = """You are an expert statistician and data scientist.
        
Your task is to provide an overall summary of a Generalized Additive Model (GAM). The model consists of different graphs that contain the effect of a specific input feature.

You will be given:
    - The global feature importances of the different features in the model.
    - Summaries of the graphs for the different features in the model. There is exactly one graph for each feature in the model.
"""
if dataset_description is None or dataset_description == '':
    prefix += "\n\nThese inputs will be given to you by the user."
else:
    prefix += "\n\nThe user will first provide a general description of what the dataset is about. Then you will be given the feature importance scores and the summaries of the individual features."

suffix = ""

if dataset_description is not None and len(dataset_description) > 0:
    suffix += "Human: Here is the general description of the data set\n" + dataset_description
    suffix += "\nAI: Thanks for this general description of the data set. Now please provide the global feature importance.\n"

suffix += "Human: Here are the global feature importaces.\n\n" + feature_importances + "\nAI: Thanks. Now please provide the descriptions of the different graphs."
suffix += "Human: Here are the descriptions of the different graphs.\n\n"
suffix+="""\nAI: Thanks. You can ask me questions next.
{history}
Human: {query}
AI:"""

print(prefix+suffix)

INFO: The graph of feature RoomService was simplified by 1.1%.
INFO: The graph of feature FoodCourt was simplified by 0.4%.
INFO: The graph of feature ShoppingMall was simplified by 0.4%.
INFO: The graph of feature Spa was simplified by 0.6%.
INFO: The graph of feature VRDeck was simplified by 0.6%.


NameError: name 'feature_importances' is not defined

In [ ]:
def llm2graph(   
        llm,
        ebm,
        feature_idx,
        memory,
        num_sentences=7,
        dataset_description = None,
        y_axis_description = None,
):
    prefix = """You are an expert statistician and data scientist.
You interpret global explanations produced by a generalized additive model (GAM). GAMs produce explanations in the form of graphs that contain the effect of a specific input feature.\n
"""
    if dataset_description is None or dataset_description == '':
        prefix +="""You will be given graphs from the model, and the user will ask you questions about the graphs."""
    else:
        prefix +="""The user will first provide a general description of the dataset. Then you will be given graphs from the model, and the user will ask you questions about the graphs.\n"""
    
    prefix +="""\n\nAnswer all questions to the best of your ability, combining both the data contained in the graph"""
    
    if dataset_description is not None and len(dataset_description) > 0:
        prefix +=""", the data set description you were given, and your knowledge about the real world."""
    else:
        prefix +=""" and your knowledge about the real world."""
    
    prefix +="""Graphs will be presented as a JSON object with keys representing the x-axis and values representing the y-axis. For continuous features, the keys are intervals that represent ranges where the function predicts the same value. For categorical features, each key represents a possible value that the feature can take."""
    
    if y_axis_description is not None and len(y_axis_description) > 0:
        prefix +=y_axis_description
    prefix += """\n\nThe user will provide graphs in the following format:
        - The name of the feature depicted in the graph
        - The type of the feature (continuous, categorical, or boolean)
        - Mean values
        - Lower bounds of confidence interval
        - Upper bounds of confidence interval
    """
    graph = t2ebm.graphs.extract_graph(ebm, feature_idx)
    graph = t2ebm.graphs.graph_to_text(graph)
    graph = graph.replace("{", "(").replace("}", ")")

    suffix="""\nBegin!
Human: Consider the following graph from the model.\n"""
    suffix+=graph
    suffix+="""\nAI: I have obtained the information of the graph. You can ask me questions next, and I will answer based on the information,my knowledge about the real world, and maybe the data description.
{history}
Human: {query}
AI:"""
    template=prefix+suffix
    prompt = PromptTemplate(input_variables=["history","query"], template=template)
    
    chain = LLMChain(
    llm = llm,
    prompt=prompt,
    verbose=True,
    memory=memory,
    )      
    return chain

In [ ]:
def llm2ebm(   #用来测试，不执行
        llm,
        ebm,
        feature_idx,
        memory,
        num_sentences=7,
        dataset_description = None,
        y_axis_description = None,
        query = "",
):
    prefix = """You are an expert statistician and data scientist.
You interpret global explanations produced by a generalized additive model (GAM). GAMs produce explanations in the form of graphs that contain the effect of a specific input feature.
"""
if dataset_description is None or dataset_description == '':
    prefix += """You will be given graphs from the model, and the user will ask you questions about the graphs."""
else:
    prefix +="""The user will first provide a general description of the dataset. Then you will be given graphs from the model, and the user will ask you questions about the graphs.\n"""

prefix += """\nAnswer all questions to the best of your ability, combining both the data contained in the graph"""

if dataset_description is not None and len(dataset_description) > 0:
    prefix +=""", the data set description you were given, and your knowledge about the real world."""
else:
    prefix +=""" and your knowledge about the real world."""

prefix +="""Graphs will be presented as a JSON object with keys representing the x-axis and values representing the y-axis. For continuous features, the keys are intervals that represent ranges where the function predicts the same value. For categorical features, each key represents a possible value that the feature can take."""

if y_axis_description is not None and len(y_axis_description) > 0:
    prefix +="""{y_axis_description}"""
prefix += """\nThe user will provide graphs in the following format:
    - The name of the feature depicted in the graph
    - The type of the feature (continuous, categorical, or boolean)
    - Mean values
    - Lower bounds of confidence interval
    - Upper bounds of confidence interval
"""

    graph = t2ebm.graphs.extract_graph(ebm, feature_idx)
    graph = t2ebm.graphs.graph_to_text(graph)
    graph = graph.replace("{", "(").replace("}", ")")

    suffix="""\nBegin!
Human: Consider the following graph from the model.\n"""
    suffix+=graph
    suffix+="""\nAI: I have obtained the information of the graph. You can ask me questions next, and I will answer based on the information,my knowledge about the real world, and maybe the data description.
{history}
Human: {query}
AI:"""
    template=prefix+suffix
    prompt = PromptTemplate(input_variables=["history","query"], template=template)
    print(template)
    chain = LLMChain(
    llm = llm,
    prompt=prompt,
    verbose=True,
    memory=memory,
    )      
    print(chain.run(query=query))

In [ ]:
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
"""You are an expert statistician and data scientist.
You interpret global explanations produced by a generalized additive model (GAM). GAMs produce explanations in the form of graphs that contain the effect of a specific input feature.
You will be given graphs from the model, and the user will ask you questions about the graphs.
Answer all questions to the best of your ability, combining both the data contained in the graph and your knowledge about the real world.Graphs will be presented as a JSON object with keys representing the x-axis and values representing the y-axis. For continuous features, the keys are intervals that represent ranges where the function predicts the same value. For categorical features, each key represents a possible value that the feature can take.
The user will provide graphs in the following format:
    - The name of the feature depicted in the graph
    - The type of the feature (continuous, categorical, or boolean)
    - Mean values
    - Lower bounds of confidence interval
    - Upper bounds of confidence interval
    """
        ),
        # The `variable_name` here is what must align with memory
        MessagesPlaceholder(variable_name="history"),
        HumanMessagePromptTemplate.from_template("""
Consider the following graph from the model.
{graph}
Please describe the general pattern of the graph at most 7 sentence.
""")
    ]
)
memory = ConversationBufferMemory(memory_key="history",return_messages=True)  

In [8]:
prompt_template = "What is a good name for a company that makes {product}?"

llm = OpenAI(temperature=0)
llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template)
)
result = llm_chain.run("colorful socks")

In [14]:
llm_chain.run("colorful socks")


'\n\nSocktastic!'

In [66]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder


In [73]:

template = """You are a chatbot having a conversation with a human.
{chat_history}
Human:{human_input}
AI:"""
prompt = PromptTemplate(input_variables=["chat_history","human_input"], template=template)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [74]:
llm = ChatOpenAI()

chat_llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory,
)

In [78]:
chat_llm_chain.predict(human_input="Answer briefly. What are the first 3 colors of a rainbow?")



> Entering new LLMChain chain...
Prompt after formatting:
You are a chatbot having a conversation with a human.
[HumanMessage(content='Hi there my friend', additional_kwargs={}, example=False), AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, example=False), HumanMessage(content='Not too bad - how are you?', additional_kwargs={}, example=False), AIMessage(content="I'm an AI chatbot, so I don't have feelings, but I'm here to assist you. How can I help you today?", additional_kwargs={}, example=False)]
Human:Answer briefly. What are the first 3 colors of a rainbow?
AI:

> Finished chain.


'The first three colors of a rainbow are red, orange, and yellow.'

In [76]:
print(memory)

chat_memory=ChatMessageHistory(messages=[HumanMessage(content='Hi there my friend', additional_kwargs={}, example=False), AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, example=False)]) output_key=None input_key=None return_messages=True human_prefix='Human' ai_prefix='AI' memory_key='chat_history'


In [79]:
chat_llm_chain.predict(human_input="And the next 4?")



> Entering new LLMChain chain...
Prompt after formatting:
You are a chatbot having a conversation with a human.
[HumanMessage(content='Hi there my friend', additional_kwargs={}, example=False), AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, example=False), HumanMessage(content='Not too bad - how are you?', additional_kwargs={}, example=False), AIMessage(content="I'm an AI chatbot, so I don't have feelings, but I'm here to assist you. How can I help you today?", additional_kwargs={}, example=False), HumanMessage(content='Answer briefly. What are the first 3 colors of a rainbow?', additional_kwargs={}, example=False), AIMessage(content='The first three colors of a rainbow are red, orange, and yellow.', additional_kwargs={}, example=False)]
Human:And the next 4?
AI:

> Finished chain.


'The next four colors of a rainbow are green, blue, indigo, and violet.'